<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Line_Disconnects_Device_Group_Phones_Same_Day_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [64]:
# Read a comma-separated values (csv) file into a Pandas DataFrame.
fact_table = pd.read_csv('drive/My Drive/Churn/discos_device_grp_fact_table_phone_vol_invol_same_day_jun.csv')

In [65]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,vol_port_basic_cdma,vol_port_basic_non_cdma,vol_port_sp_android_other,vol_port_sp_apple,vol_non_port_basic_cdma,vol_non_port_basic_non_cdma,vol_non_port_sp_android_other,vol_non_port_sp_apple,invol_basic_cdma,invol_basic_non_cdma,invol_sp_android_other,invol_sp_apple
0,12/1/2016,218,NaN,961,1034,1415,NaN,1502,1152,411,NaN,2126,1731
1,12/2/2016,197,NaN,1192,1117,1367,NaN,1989,1076,269,NaN,1491,1312
2,12/3/2016,268,NaN,1357,1566,921,NaN,846,784,263,NaN,1728,1370
3,12/4/2016,239,NaN,925,1356,534,NaN,593,525,239,NaN,1226,1007
4,12/5/2016,198,NaN,1088,1154,1533,NaN,1412,1129,307,NaN,1960,1456


In [66]:
# Split out indivudal datasets for forecasting.
# Voluntary Port Outs
vol_port_basic_cdma = fact_table[['ds', 'vol_port_basic_cdma']].rename(columns = {'vol_port_basic_cdma': 'y'})
vol_port_basic_non_cdma = fact_table[['ds', 'vol_port_basic_non_cdma']].rename(columns = {'vol_port_basic_non_cdma': 'y'})
vol_port_sp_android_other = fact_table[['ds', 'vol_port_sp_android_other']].rename(columns = {'vol_port_sp_android_other': 'y'})
vol_port_sp_apple = fact_table[['ds', 'vol_port_sp_apple']].rename(columns = {'vol_port_sp_apple': 'y'})

# Voluntary non Port Outs
vol_non_port_basic_cdma = fact_table[['ds', 'vol_non_port_basic_cdma']].rename(columns = {'vol_non_port_basic_cdma': 'y'})
vol_non_port_basic_non_cdma = fact_table[['ds', 'vol_non_port_basic_non_cdma']].rename(columns = {'vol_non_port_basic_non_cdma': 'y'})
vol_non_port_sp_android_other = fact_table[['ds', 'vol_non_port_sp_android_other']].rename(columns = {'vol_non_port_sp_android_other': 'y'})
vol_non_port_sp_apple = fact_table[['ds', 'vol_non_port_sp_apple']].rename(columns = {'vol_non_port_sp_apple': 'y'})

# Involuntary
invol_basic_cdma = fact_table[['ds', 'invol_basic_cdma']].rename(columns = {'invol_basic_cdma': 'y'})
invol_basic_non_cdma = fact_table[['ds', 'invol_basic_non_cdma']].rename(columns = {'invol_basic_non_cdma': 'y'})
invol_sp_android_other = fact_table[['ds', 'invol_sp_android_other']].rename(columns = {'invol_sp_android_other': 'y'})
invol_sp_apple = fact_table[['ds', 'invol_sp_apple']].rename(columns = {'invol_sp_apple': 'y'})

In [67]:
# Check the individual dataset format (testing only)
vol_port_basic_cdma.head()

,ds,y
0,12/1/2016,218
1,12/2/2016,197
2,12/3/2016,268
3,12/4/2016,239
4,12/5/2016,198


In [68]:
# Define the holidays
new_years_day = pd.DataFrame({
    'holiday': 'new_years_day',
    'ds': pd.to_datetime(['2017-01-01',
                          '2018-01-01',
                          '2019-01-01',
                          '2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

mlk_day = pd.DataFrame({
    'holiday': 'mlk_day',
    'ds': pd.to_datetime(['2017-01-16',
                          '2018-01-15',
                          '2019-01-21',
                          '2020-01-20']),
    'lower_window': 0,
    'upper_window': 0,
})

easter_sunday = pd.DataFrame({
    'holiday': 'easter_sunday',
    'ds': pd.to_datetime(['2017-04-16',
                          '2018-04-01',
                          '2019-04-21',
                          '2020-04-12']),
    'lower_window': 0,
    'upper_window': 0,
})

memorial_day = pd.DataFrame({
    'holiday': 'memorial_day',
    'ds': pd.to_datetime(['2017-05-29',
                          '2018-05-28',
                          '2019-05-27',
                          '2020-05-25']),
    'lower_window': 0,
    'upper_window': 0,
})

independence_day = pd.DataFrame({
    'holiday': 'independence_day',
    'ds': pd.to_datetime(['2017-07-04',
                          '2018-07-04',
                          '2019-07-04',
                          '2020-07-04']),
    'lower_window': 0,
    'upper_window': 0,
})

labor_day = pd.DataFrame({
    'holiday': 'labor_day',
    'ds': pd.to_datetime(['2017-09-04',
                          '2018-09-03',
                          '2019-09-02',
                          '2020-09-07']),
    'lower_window': 0,
    'upper_window': 0,
})

thanksgiving_day = pd.DataFrame({
    'holiday': 'thanksgiving_day',
    'ds': pd.to_datetime(['2017-11-23',
                          '2018-11-22',
                          '2019-11-28',
                          '2020-11-26']),
    'lower_window': 0,
    'upper_window': 0,
})

black_friday = pd.DataFrame({
    'holiday': 'black_friday',
    'ds': pd.to_datetime(['2017-11-24',
                          '2018-11-23',
                          '2019-11-29',
                          '2020-11-27']),
    'lower_window': 0,
    'upper_window': 0,
})

cyber_monday = pd.DataFrame({
    'holiday': 'cyber_monday',
    'ds': pd.to_datetime(['2017-11-27',
                          '2018-11-26',
                          '2019-12-02',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

christmas_day = pd.DataFrame({
    'holiday': 'christmas_day',
    'ds': pd.to_datetime(['2017-12-25',
                          '2018-12-25',
                          '2019-12-25',
                          '2020-12-25']),
    'lower_window': 0,
    'upper_window': 0,
})

In [69]:
holidays = pd.concat((new_years_day,
                      mlk_day,
                      easter_sunday,
                      memorial_day,
                      independence_day,
                      labor_day,
                      thanksgiving_day,
                      black_friday,
                      cyber_monday,
                      christmas_day))

In [70]:
# Define the models
# Voluntary Port Outs
vol_port_basic_cdma_model = Prophet(holidays = holidays)
vol_port_basic_non_cdma_model = Prophet(holidays = holidays)
vol_port_sp_android_other_model = Prophet(holidays = holidays)
vol_port_sp_apple_model = Prophet(holidays = holidays)

# Voluntary non Port Outs
vol_non_port_basic_cdma_model = Prophet(holidays = holidays)
vol_non_port_basic_non_cdma_model = Prophet(holidays = holidays)
vol_non_port_sp_android_other_model = Prophet(holidays = holidays)
vol_non_port_sp_apple_model = Prophet(holidays = holidays)

# Involuntary 
invol_basic_cdma_model = Prophet(holidays = holidays)
invol_basic_non_cdma_model = Prophet(holidays = holidays)
invol_sp_android_other_model = Prophet(holidays = holidays)
invol_sp_apple_model = Prophet(holidays = holidays)

In [72]:
# Add seasonality
# Voluntary Port Outs
vol_port_basic_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_port_basic_non_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_port_sp_android_other_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_port_sp_apple_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)

# Voluntary non Port Outs
vol_non_port_basic_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_non_port_basic_non_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_non_port_sp_android_other_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
vol_non_port_sp_apple_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)

# Involuntary 
invol_basic_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
invol_basic_non_cdma_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
invol_sp_android_other_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)
invol_sp_apple_model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)

In [73]:
# Fit the models
# Voluntary Port Outs
vol_port_basic_cdma_model.fit(vol_port_basic_cdma)
vol_port_basic_non_cdma_model.fit(vol_port_basic_non_cdma)
vol_port_sp_android_other_model.fit(vol_port_sp_android_other)
vol_port_sp_apple_model.fit(vol_port_sp_apple)

# Voluntary non Port Outs
vol_non_port_basic_cdma_model.fit(vol_non_port_basic_cdma)
vol_non_port_basic_non_cdma_model.fit(vol_non_port_basic_non_cdma)
vol_non_port_sp_android_other_model.fit(vol_non_port_sp_android_other)
vol_non_port_sp_apple_model.fit(vol_non_port_sp_apple)

# Involuntary 
invol_basic_cdma_model.fit(invol_basic_cdma)
invol_basic_non_cdma_model.fit(invol_basic_non_cdma)
invol_sp_android_other_model.fit(invol_sp_android_other)
invol_sp_apple_model.fit(invol_sp_apple)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [74]:
# Number of periods into the future to forecast
n = 184

In [75]:
# Create dataframes to hold predictions
# Voluntary Port Outs
vol_port_basic_cdma_future = vol_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_port_basic_non_cdma_future = vol_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_port_sp_android_other_future = vol_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_port_sp_apple_future = vol_port_sp_apple_model.make_future_dataframe(periods = n)

# Voluntary non Port Outs
vol_non_port_basic_cdma_future = vol_non_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_non_cdma_future = vol_non_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_non_port_sp_android_other_future = vol_non_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_non_port_sp_apple_future = vol_non_port_sp_apple_model.make_future_dataframe(periods = n)

# Involuntary
invol_basic_cdma_future = invol_basic_cdma_model.make_future_dataframe(periods = n)
invol_basic_non_cdma_future = invol_basic_non_cdma_model.make_future_dataframe(periods = n)
invol_sp_android_other_future = invol_sp_android_other_model.make_future_dataframe(periods = n)
invol_sp_apple_future = invol_sp_apple_model.make_future_dataframe(periods = n)

In [76]:
# Make predictions
# Voluntary Port Outs
vol_port_basic_cdma_forecast = vol_port_basic_cdma_model.predict(vol_port_basic_cdma_future)
vol_port_basic_non_cdma_forecast = vol_port_basic_non_cdma_model.predict(vol_port_basic_non_cdma_future)
vol_port_sp_android_other_forecast = vol_port_sp_android_other_model.predict(vol_port_sp_android_other_future)
vol_port_sp_apple_forecast = vol_port_sp_apple_model.predict(vol_port_sp_apple_future)

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast = vol_non_port_basic_cdma_model.predict(vol_non_port_basic_cdma_future)
vol_non_port_basic_non_cdma_forecast = vol_non_port_basic_non_cdma_model.predict(vol_non_port_basic_non_cdma_future)
vol_non_port_sp_android_other_forecast = vol_non_port_sp_android_other_model.predict(vol_non_port_sp_android_other_future)
vol_non_port_sp_apple_forecast = vol_non_port_sp_apple_model.predict(vol_non_port_sp_apple_future)

# Involuntary
invol_basic_cdma_forecast = invol_basic_cdma_model.predict(invol_basic_cdma_future)
invol_basic_non_cdma_forecast = invol_basic_non_cdma_model.predict(invol_basic_non_cdma_future)
invol_sp_android_other_forecast = invol_sp_android_other_model.predict(invol_sp_android_other_future)
invol_sp_apple_forecast = invol_sp_apple_model.predict(invol_sp_apple_future)

In [77]:
# Get the datestamp and prediction columns
# Voluntary Port Outs
vol_port_basic_cdma_forecast_subset = vol_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_CDMA'})
vol_port_basic_non_cdma_forecast_subset = vol_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_non_CDMA'})
vol_port_sp_android_other_forecast_subset = vol_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Android_Other'})
vol_port_sp_apple_forecast_subset = vol_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Apple'})

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast_subset = vol_non_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_CDMA'})
vol_non_port_basic_non_cdma_forecast_subset = vol_non_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_non_CDMA'})
vol_non_port_sp_android_other_forecast_subset = vol_non_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Android_Other'})
vol_non_port_sp_apple_forecast_subset = vol_non_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Apple'})

# Involuntary
invol_basic_cdma_forecast_subset = invol_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_CDMA'})
invol_basic_non_cdma_forecast_subset = invol_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_non_CDMA'})
invol_sp_android_other_forecast_subset = invol_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Android_Other'})
invol_sp_apple_forecast_subset = invol_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Apple'})

In [78]:
# Check the format of the subsetted forecasts (testing only)
vol_port_basic_cdma_forecast_subset.tail()

,ds,Vol_Port_Basic_CDMA
1487,2020-12-27,44.272012
1488,2020-12-28,79.550874
1489,2020-12-29,75.066539
1490,2020-12-30,70.412974
1491,2020-12-31,60.935597


In [79]:
# Growth multiplier
gm = 1.00

In [80]:
# Replace any negative values and scale the forecast using the growth multiplier
# Voluntary Port Outs 
vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'] = vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'].abs() * gm
vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'] = vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'].abs() * gm
vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'] = vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'].abs() * gm
vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'] = vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'].abs() * gm

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'] = vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'].abs() * gm
vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'] = vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'].abs() * gm
vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'] = vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'].abs() * gm
vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'] = vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'].abs() * gm

# Involuntary
invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'] = invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'].abs() * gm
invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'] = invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'].abs() * gm
invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'] = invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'].abs() * gm
invol_sp_apple_forecast_subset['Invol_SP_Apple'] = invol_sp_apple_forecast_subset['Invol_SP_Apple'].abs() * gm

In [81]:
# Check the format of the adjusted forecasts (testing only)
vol_port_sp_apple_forecast_subset.tail()

,ds,Vol_Port_SP_Apple
1487,2020-12-27,604.890140
1488,2020-12-28,752.548508
1489,2020-12-29,669.052253
1490,2020-12-30,626.121062
1491,2020-12-31,545.259661


In [82]:
# Combine the forecasts into one table
# Voluntary Port Outs
discos_device_grp_phone_pred_table = vol_port_basic_cdma_forecast_subset
discos_device_grp_phone_pred_table.insert(2, 'Vol_Port_Basic_non_CDMA', vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(3, 'Vol_Port_SP_Android_Other', vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(4, 'Vol_Port_SP_Apple', vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'])

# Voluntary Non Port Outs
discos_device_grp_phone_pred_table.insert(5, 'Vol_Non_Port_Basic_CDMA', vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'])
discos_device_grp_phone_pred_table.insert(6, 'Vol_Non_Port_Basic_non_CDMA', vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(7, 'Vol_Non_Port_SP_Android_Other', vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(8, 'Vol_Non_Port_SP_Apple', vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'])

# Involuntary
discos_device_grp_phone_pred_table.insert(9, 'Invol_Basic_CDMA', invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'])
discos_device_grp_phone_pred_table.insert(10, 'Invol_Basic_non_CDMA', invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(11, 'Invol_SP_Android_Other', invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(12, 'Invol_SP_Apple', invol_sp_apple_forecast_subset['Invol_SP_Apple'])

In [83]:
# Check the format of the final predictions table (testing only)
discos_device_grp_phone_pred_table.tail()

,ds,Vol_Port_Basic_CDMA,Vol_Port_Basic_non_CDMA,Vol_Port_SP_Android_Other,Vol_Port_SP_Apple,Vol_Non_Port_Basic_CDMA,Vol_Non_Port_Basic_non_CDMA,Vol_Non_Port_SP_Android_Other,Vol_Non_Port_SP_Apple,Invol_Basic_CDMA,Invol_Basic_non_CDMA,Invol_SP_Android_Other,Invol_SP_Apple
1487,2020-12-27,44.272012,18.862850,245.454826,604.890140,151.437162,169.985009,144.156704,657.955498,20.123929,69.770051,674.352861,452.902667
1488,2020-12-28,79.550874,25.092570,382.878598,752.548508,523.633119,256.063982,902.192774,1479.325317,15.425919,61.378219,629.189026,410.352992
1489,2020-12-29,75.066539,23.924327,338.445033,669.052253,464.437069,256.436950,829.655273,1394.189719,10.689919,61.984608,829.129673,632.522270
1490,2020-12-30,70.412974,23.173120,324.498999,626.121062,426.681300,249.628354,789.545750,1331.575512,59.978676,66.318027,1053.552342,805.313300
1491,2020-12-31,60.935597,22.528869,292.537490,545.259661,418.326072,251.333441,787.018772,1325.658233,78.163611,76.947047,1057.268041,844.756262


In [84]:
# Calculate the root mean squared error (testing only)
print("Vol Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_phone_pred_table.loc[:1307, 'Vol_Port_Basic_CDMA'] - fact_table['vol_port_basic_cdma']) ** 2)))
print("Vol Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_phone_pred_table.loc[:1307, 'Vol_Port_Basic_non_CDMA'] - fact_table['vol_port_basic_non_cdma']) ** 2)))
print("Vol Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_phone_pred_table.loc[:1307, 'Vol_Port_SP_Android_Other'] - fact_table['vol_port_sp_android_other']) ** 2)))
print("Vol Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_phone_pred_table.loc[:1307, 'Vol_Port_SP_Apple'] - fact_table['vol_port_sp_apple']) ** 2)))

Vol Port Basic CDMA RMSE: 25.761902
Vol Port Basic Non CDMA RMSE: 4.281291
Vol Port SP Android Other RMSE: 108.461777
Vol Port SP Apple RMSE: 167.216404


In [85]:
# Save the predictions table to a csv file
discos_device_grp_phone_pred_table.to_csv('drive/My Drive/Churn/discos_device_grp_phone_same_day_pred_table.csv')